In [2]:
import os
import numpy as np
import pandas as pd
from time import time

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams["font.size"] = 26

data_dir = "/home/stavros/DATA/AirbnbReviews"
#data_dir = "D:/"

In [10]:
area = "nyc"
area_dir = os.path.join(data_dir, area)

reviews = pd.read_csv(os.path.join(area_dir, "nyc_reviews_20000samplesnostopwords.csv"))

In [11]:
print(reviews.shape)
print(reviews.columns)

(20000, 8)
Index(['Unnamed: 0', 'listing_id', 'id', 'date', 'reviewer_id',
       'reviewer_name', 'comments', 'normalized_comments'],
      dtype='object')


In [15]:
clean_reviews = reviews[pd.notnull(reviews.normalized_comments)]
clean_reviews.shape

(19820, 8)

Tokenize the documents

In [50]:
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
docs = [tokenizer.tokenize(review) for review in clean_reviews.normalized_comments]
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

Create dictionary and bag-of-words corpus

In [51]:
from gensim import corpora, models
dictionary = corpora.Dictionary(docs)
print(len(dictionary))
dictionary.filter_extremes(no_below=10, no_above=0.1)
print(len(dictionary))

25849
3639


In [52]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Apply `gensim` LDA model

In [53]:
_ = dictionary[0]
id2word = dictionary.id2token

model = models.LdaModel(corpus=corpus, id2word=dictionary.id2token, num_topics=3, passes=20)

In [55]:
model.print_topics()

[(0,
  '0.046*"de" + 0.032*"la" + 0.023*"et" + 0.020*"en" + 0.019*"un" + 0.017*"est" + 0.017*"tre" + 0.016*"que" + 0.016*"el" + 0.015*"muy"'),
 (1,
  '0.036*"und" + 0.027*"die" + 0.023*"ist" + 0.021*"cancel" + 0.021*"reservation" + 0.021*"arrival" + 0.020*"day" + 0.019*"sehr" + 0.018*"posting" + 0.017*"automate"'),
 (2,
  '0.008*"home" + 0.007*"also" + 0.007*"check" + 0.007*"bed" + 0.007*"neighborhood" + 0.007*"space" + 0.006*"area" + 0.006*"like" + 0.006*"go" + 0.006*"helpful"')]